In [107]:
from login_details import engine1

In [129]:
with engine1.connect() as con:
    query = con.execute("""
    SELECT id, stoppage_id, datetime
    FROM dg_ln_stoppages_raw_1
    ORDER BY datetime ASC
    LIMIT 10000
    """)



In [130]:
# setting value for the first iteration of the variable previous_row
previous_row = -1   # -1 only indicates that the variable hasn't been set - first iteration
list_of_faults_keep = []
list_of_faults_delete = []
keep = []

# iterate through the database rows
for row in query:
    # previous_row needs a value for first iteration. After this, the value -1 will be overwritten by the actual row value
    if previous_row == -1:  # first iteration
        previous_row = row[1]  # save the FAULT ID value
        list_of_faults_keep.append(row[1])

        # for testing
        fresh_list = [row]
        keep.append(fresh_list)  # save the row ID into the keep list
        fresh_list = []

    elif row[1] == previous_row:  # repeating value - needs to be deleted
        list_of_faults_delete.append(row[0])  # save the row ID into the delete list
        previous_row = row[1]  # save the FAULT ID value

    elif row[1] != previous_row:  # the value is different from the previous row (change of state)
        if row[1] in list_of_faults_keep:
            if row[1] == 0 and len(list_of_faults_keep) > 1:
                if list_of_faults_delete:  # if the delete list is empty, reset the keep list. This is to get past already sorted rows
                    break

                else:
                    list_of_faults_keep = []


            list_of_faults_delete.append(row[0])
            previous_row = row[1]  # save the FAULT ID value

        elif row[1] not in list_of_faults_keep:
            list_of_faults_keep.append(row[1])
            # for testing
            fresh_list = [row]
            keep.append(fresh_list)  # save the row ID into the keep list
            fresh_list = []
            previous_row = row[1]  # save the FAULT ID value

In [133]:
print("Keep List:\n")
print('\n'.join(map(str, list_of_faults_keep)))

Keep List:

0
64
49
1
59
44
69
42


In [132]:
list_of_faults_delete = tuple(list_of_faults_delete)

params = {'l': list_of_faults_delete}

print(params)


{'l': ()}


In [128]:
with engine1.connect() as con:
            delete_statement = con.execute("""
            DELETE FROM dg_ln_stoppages_raw_1
            WHERE id in %(l)s;
            """, params)
            print('Done')

Done


In [123]:
list_of_faults_delete = []